# [Muscle-bids](https://pypi.org/project/muscle-bids): dcm2mbids

By: Leonardo Barzaghi, [Donnie Cameron](https://github.com/DC-3T), Judith Cueto Fernandez, Jilmen Quintiens, [Francesco Santini](https://github.com/fsantini), 2022
Version 1.0  
Code licence: GPL v3  
Narrative licence: CC-BY  
How to cite: (TBD)

---
---

## Aims

The [muscle-bids package](https://github.com/muscle-bids/muscle-bids/tree/Jupyter) converts traditional musculoskeletal MRI data in DICOM format into a structure compatible with [BIDS](https://bids.neuroimaging.io/)[<sup id="fn1-back">1</sup>](#fn1). It can be run as a standalone executable or called within Python scripts as a library.

In this notebook you will learn:

1. How to install `muscle-bids`; and

2. How to use `muscle-bids` as either an executable or as a Python module.

### Table of contents

[Installation](#Installation)  
[Usage examples](#Usage-examples)  
[Dependencies](#Dependencies)  
[References](#References)  
[Acknowledgements](#Acknowledgements)
    

---

## Installation

To install the muscle-bids package, run the code below, noting the dependencies at the bottom of this document.

First, we clone the git repository:

In [ ]:
!git clone https://github.com/muscle-bids/muscle-bids.git

Now we can install the package using `pip`. This will also install the required dependencies.

In [ ]:
%cd muscle-bids
!pip install .
!pip install --upgrade nibabel # the default nibabel has bugs

The package is now ready to be used! Lets decompress the example files.

In [ ]:
!unzip dicom.zip

Let's have a look at what data are available. We'll use the Python package `directory_tree`, which allows us to display our directory structure.

In [ ]:
!pip install directory_tree

In [ ]:
from directory_tree import display_tree
display_tree('dicom')

These are some multi-echo gradient-echo data from GE, and some multi-echo spin-echo data from Philips.
You can browse these files inside the dicom directory from the file explorer. You will notice that the GE data comprise multiple DICOM files, whereas Philips data are contained in a single, large file.

You are now ready to start using the muscle-bids package to standardise your data according to BIDS!

---

## Usage examples

We will now demonstrate the usage of `muscle-bids` via two use cases: 
    
1. Running `dcm2mbids` as an executable to convert DICOM data to the BIDS format.
2. Importing `muscle-bids` as a Python module to find, load, and interrogate BIDS-format data.
    


### 1. Usage: Run as executable

The Python package is set up in such a way that, after installing, you can use the code in your own code, and directly from the command line as well.

The commandline script is called `dcm2mbids.exe` and is by default stored in the same folder of your Python executable (and should be added to your system's PATH variable). 

Let's look at how we can call it.


In [ ]:
!dcm2mbids -h

In principle, the program should be able to automatically figure out what the data are. Let's see what happens when we run `dcm2bids` recursively in our root folder:

In [ ]:
!dcm2mbids -a mypatient -r dicom bids_output

The output data are saved to a new directory called `bids_output`. Let's see what's inside:

In [ ]:
display_tree('bids_output')

You can see that the converted data are sorted under `anat` for anatomical images and `quant` for quantitative maps. All the maps and MESE images were previously condensed in a single Enhanced DICOM file by the Philips scanner.

Every dataset has a nii.gz file containing the image data, and a set of json files containing the headers. Specifically:


*   The simple `.json` file contains useful information about the data
*   The `_patient.json` file contains private patient data. Delete this file if you want to anonymise your data!
*   The `_extra.json` file contains extra information that can be used to reconstruct the DICOM dataset from the BIDS data, so it can be stored again in PACS.



  ### 2. Usage: Import as python module

The muscle-bids package can also be used as an imported module within your Python scripts. Ideally, when you program an image processing tool, you will want to use this particular BIDS format for both input and output. You just need to point your Python script to the root folder of your patient information.

First, let's __import__ the required modules:

In [ ]:
from muscle_bids.utils.io import find_bids, load_bids
import nibabel as nib 

Now, let's - very simply - load some multi-echo spin-echo (MESE) data.

Let's see if we can __find__ the MESE BIDS data first: 

In [ ]:
BIDS_FOLDER = 'bids_output'

mese_data_list = find_bids(BIDS_FOLDER, 'mese')

print(mese_data_list)

Good news! There's at least one BIDS dataset with MESE data! Let's __load__ those data:

In [ ]:
bids_mese = load_bids(mese_data_list[0])
nib.viewers.OrthoSlicer3D(bids_mese)

The BIDS file also has three dictionaries associated with it:

*   `bids_header`: containing relevant information
*   `patient_header`: containing privacy-relevant information (missing if anonymized)
*   `extra_header`: a dump of the other DICOM tags

Let's have a look, for example, at the BIDS header.


In [ ]:
bids_mese.bids_header

You can now use the BIDS header information in tandem with your data (accessible from `bids_mese.volume`) to do any further processing, and you can later save the processed output in BIDS format, or as DICOM files.

---
## Dependencies

In [ ]:
!pip install watermark

In [ ]:
import muscle_bids  # Import full package to avoid garbled watermark output

In [ ]:
%load_ext watermark

%watermark
%watermark --iversions

---
## References
[<sup id="fn1">1</sup>](#fn1-back) Gorgolewski, K., Auer, T., Calhoun, V. et al. The brain imaging data structure, a format for organizing and describing outputs of neuroimaging experiments. Sci Data 3, 160044 (2016). https://doi.org/10.1038/sdata.2016.44

---
## Acknowledgements
This package was developed with the support of the ORMIR community during the Jupyter Community Workshop and hackathon in Maastricht, The Netherlands, in June 2022.  

Image I/O is based on [DOSMA](https://github.com/ad12/DOSMA) by Arjun Desai. A stripped-down version of DOSMA is present in muscle-bids/dosma_io.  

Notebook created using the [ORMIR community](https://ormircommunity.github.io/) template (version 1.0, 2023).